to do:
- image in tensorboard (test alpha mobilenet)
- imports aufräumen
- Secondstage.py
- Konfiguration außerhalb:
    - Name output
    - cat/ bin/ reg
- endliche Anzahl an Runden

In [0]:
%tensorflow_version 1.x

object_detection


In [0]:
!git clone --quiet https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


In [0]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools

In [0]:
%cd models/research

[Errno 2] No such file or directory: 'models/research'
/content/models/research


In [0]:
!protoc object_detection/protos/*.proto --python_out=.

object_detection/protos/*.proto: No such file or directory


In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

In [0]:
!python object_detection/builders/model_builder_test.py

python3: can't open file 'object_detection/builders/model_builder_test.py': [Errno 2] No such file or directory


In [0]:
from google.colab import drive, files
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/SecondStage_colab/')

In [0]:
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/SecondStage_colab/'

Second Stage

In [0]:
#NAME =
#example: cat_exptype_dataset(training)_dataset(eval)_conf(batch size etc) 

In [0]:
LOG_PATH = '/content/gdrive/My Drive/SecondStage_colab/output/'
LABEL_MAP_PATH = '/content/gdrive/My Drive/SecondStage_colab/label_map.pbtxt'
TRAIN_RECORD = '/content/gdrive/My Drive/data/generated/SecondStage_X/training_rot3.record'
EVAL_RECORD = '/content/gdrive/My Drive/data/generated/SecondStage_X/validation_rot2.record'

In [0]:
#TRAIN_RECORD = 'training_rot15.record'
#EVAL_RECORD = 'validation_rot2.record'

In [0]:
os.chdir('/content/gdrive/My Drive/')
!ls
from SecondStage_colab import second_stage_utils
from SecondStage_colab import file_utils

'8-Week General Cycle.md'
'Anmeldung Bierpongturnier (Antworten).gsheet'
'Anmeldung Bierpongturnier.gform'
 Assignment3
'BA - Rolling Swarm.gdoc'
'Colab Notebooks'
 data
'Deadlift Program.gsheet'
 DHBW_App_Projektstruktur_rev1.02.pdf
 events.out.tfevents.1579157229.5ae34ead26e8
 events.out.tfevents.1579157352.5ae34ead26e8
 events.out.tfevents.1579184572.b50e20e5cc32
 Ferienhäuser.gsheet
'HSPU Progression.gdoc'
'HSPU Progression.gsheet'
 Klim.gdoc
'Kopie von Links 1. Semester (1).gsheet'
'Kopie von Links 1. Semester.gsheet'
'Kündigung CFRN.gdoc'
'Kündigung EVG.gdoc'
'Kündigung Fitness Schleusingen.gdoc'
'Kündigung - Zeller.gdoc'
 Laptops.gsheet
'Links 1. Semester.gsheet'
 object_detection
'Ohne Titel.gdoc'
'Pflichtenheft – Team EEG.gdoc'
'Projektablauf und -strutkurplan.gsheet'
 Risiko.gsheet
 Second_Stage
 SecondStage_colab
'TechInf 1.gdoc'
 TechInf1.gdoc
'TechInf 2.gdoc'
'TechInf 3.gdoc'
'TechInf 5.gdoc'
'TI2 - Blatt 06.gdoc'
'TI2 - Blatt2.gdoc'
'TI2 - Blatt 4.gdoc'
'TI - Blatt 0

In [0]:
os.chdir('/content/models/research')
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

configuration

In [0]:
GPU = True
BATCH_SIZE = 7425



label map


In [0]:
os.chdir('/content/gdrive/My Drive/SecondStage_colab')

In [0]:
%%writefile label_map.pbtxt

item {
  id: 1
  name: 'red'
}

item {
  id: 2
  name: 'orange'
}

item {
  id: 3
  name: 'yellow'
}

item {
  id: 4
  name: 'lime_green'
}

item {
  id: 5
  name: 'magenta' 
}

item {
  id: 6
  name: 'purple'
}

item {
  id: 7
  name: 'green'
}

item {
  id: 8
  name: 'light_green'
}

item {
  id: 9
  name: 'blue_green'
}

item {
  id: 10
  name: 'light_blue'
}

item {
  id: 11
  name: 'blue'
}

Overwriting label_map.pbtxt


experiment definitioin


In [0]:
%%writefile exp_def.py

from copy import deepcopy

default_sstage_conf = {
    'dataset': 'default',
    'types': ['copter','sphero','youbot'],
    # converged after 15 / 10 / 5 epochs
    # 10 epochs after convergance for evaluation phase
    'epochs_cat': 60,
    #'epochs_reg': 20,
    'epochs_reg': 60,
    'epochs_bin': 60,
    'optimizer': 'adam',
    'learning_rate': 3e-4,
    'dropout': 0,
    'alpha': 0.5,
    'img_size': 35,
    'separate_cat_ori': True,
    'cat_weight': 1.0,
    'reg_weight': 1.0,
    'bin_weight': 1.0,
    'enable_reg': False,
    'enable_bin': True,
    'repetions': 4,
}

def create_all_sstage_experiments():
    configs = []
    configs.extend(create_sstage_default())
    configs.extend(bin_mod())
    return configs

def create_sstage_default():
    config = []
    modified = deepcopy(default_sstage_conf)
    modified['name'] = "sstage_default"
    modified['enable_reg'] = True
    config.append(deepcopy(modified))
    return config

def bin_mod():
    config = []
    modified = deepcopy(default_sstage_conf)
    modified['name'] = "bin_mod"
    modified['enable_reg'] = True
    config.append(deepcopy(modified))
    return config

Overwriting exp_def.py


In [0]:
from tqdm import tqdm
import re
from datetime import datetime
from multiprocessing import Process, Queue

from copy import deepcopy

from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Reshape, Conv2D, Activation
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, Callback
from keras.models import load_model
from keras.utils import np_utils

import numpy as np
from numpy.random import randint

import tensorflow as tf
from PIL import Image
import io

import csv, json, pickle
from lxml import etree


In [0]:
def train(train_record, conf, types, out, rep=1):

    from file_utils import save_json
    from second_stage_utils import tf_record_load_crops, tf_record_extract_crops, data_to_keras, angle_to_bin, angle_mse, angle_mae, angle_bin_error, TensorBoardCustom

    timestamp = "{:%Y-%m-%d-%H-%M}".format(datetime.now())
    log_path = LOG_PATH + conf['name'] + '_' + \
                ''.join(types) + out + '/' + \
                timestamp + '-r' + str(rep) + '/'
    os.makedirs(log_path, exist_ok=True)
    save_json(log_path + '/experiment_config.json', conf)

    label_map = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH)
    #label_map = create_category_index_from_labelmap(LABEL_MAP_PATH)

    num_classes = label_map_util.get_max_label_map_index(
                           label_map_util.load_labelmap(LABEL_MAP_PATH)) + 1
    print("x")

    X,Y,Z,_ = tf_record_load_crops([train_record])

    """
    for i in range(ROT_SIZE):
      X.extend(X)
      Y.extend(Y)
      Z.extend(Z)
      _.extend(_)
    """
    
    X_train, Y_train, Z_train = data_to_keras(X,Y,Z,num_classes,conf['img_size'])
    Z2_train = angle_to_bin(Z_train)
    print("´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´")
    print(Y_train.shape)
    print(num_classes)
    print(Z2_train.shape)
    print(NUM_ORI_BINS)
    print("´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´")

    eval_records = EVAL_RECORD

    X,Y,Z,_ = tf_record_extract_crops([eval_records], 1, 0.0, 0.0)
    X_val, Y_val, Z_val = data_to_keras(X,Y,Z,num_classes,conf['img_size'])
    Z2_val = angle_to_bin(Z_val)

    assert len(X_val) > 0 and len(Y_val) > 0 and len(Z_val) > 0, \
        '{} is incomplete'.format(eval_records)
    mobilenet_base = applications.mobilenet.MobileNet(alpha = conf['alpha'],
                                                      weights = "imagenet",
                                                      include_top=False,
                                                      input_shape = (
                                                      conf['img_size'],
                                                      conf['img_size'],
                                                        3
                                                      ))
    shape = (1, 1, int(1024 * conf['alpha']))
    x = GlobalAveragePooling2D()(mobilenet_base.output)
    x = Reshape(shape, name='reshape_1')(x)
    x = Dropout(conf['dropout'], name='dropout')(x)
    # Branch regression
    reg = Conv2D(1, (1, 1),
               padding='same', name='conv_reg')(x)
    reg = Activation('linear', name='act_linear')(reg)
    reg = Reshape((1,), name='reg_out')(reg)
    # Branch orientation classification with bins
    bin = Conv2D(NUM_ORI_BINS, (1, 1),
               padding='same', name='conv_bin')(x)
    bin = Activation('softmax', name='act_bin')(bin)
    bin = Reshape((NUM_ORI_BINS,), name='bin_out')(bin)
    # Branch classification
    cat = Conv2D(num_classes, (1, 1),
               padding='same', name='conv_cat')(x)
    cat = Activation('softmax', name='act_softmax')(cat)
    cat = Reshape((num_classes,), name='cat_out')(cat)

    # creating the final model
    model_final = None
    model_final = Model(inputs = mobilenet_base.input, outputs = [cat,reg,bin])

    if conf['optimizer'] == 'rmsprop':
        optimizer = optimizers.RMSprop(lr=conf['learning_rate'])
    elif conf['optimizer'] == 'adam':
        optimizer = optimizers.Adam(lr=conf['learning_rate'])
    else:
        raise Error('Unknown optimizer: ' + conf['optimizer'])

    model_final.compile(optimizer = optimizer,
                        loss={'cat_out': 'categorical_crossentropy',
                              'reg_out': angle_mse,
                              'bin_out': 'categorical_crossentropy',
                              #'bin_out': 'binary_crossentropy'
                        },
                        loss_weights={'cat_out': conf['cat_weight'],
                                      'reg_out': conf['reg_weight'],
                                      'bin_out': conf['bin_weight']},
                        metrics ={'cat_out': 'accuracy',
                                  'reg_out': angle_mae,
                                  'bin_out': angle_bin_error})

    summary = TensorBoardCustom(log_dir=log_path,label_map=label_map)
    filepath=log_path+"model-{epoch:02d}.h5"
    checkpoint = ModelCheckpoint(filepath, verbose=1, period=20)

    """callbacks=[summary,checkpoint],"""

    model_final.fit(
        X_train,
        {'cat_out': Y_train, 'reg_out': Z_train, 'bin_out': Z2_train},
        validation_data=(X_val,[Y_val,Z_val, Z2_val]),
        batch_size=BATCH_SIZE, epochs=conf['epochs'], verbose=0,
        callbacks=[summary,checkpoint],
        shuffle=True
    )
    model_final.save(log_path+"model-final.h5")
    print("Finished training for {}_{}".format(conf['name'],t))


In [0]:
from SecondStage_colab import exp_def
from copy import deepcopy

In [0]:
ANGLES_PER_BIN = 1
NUM_ORI_BINS = 360

exp = exp_def.create_all_sstage_experiments()
print(BATCH_SIZE)
for or_conf in tqdm(exp):
    print("or_conf", or_conf)
    for r in range(or_conf['repetions']): 
        #train_records = get_recursive_file_list(TRAIN_DIR,file_matchers=[or_conf['dataset']])
        train_records = [TRAIN_RECORD]
        for t in or_conf['types']:
            for out in ['_cat','_reg','_bin','']:
                conf = deepcopy(or_conf)
                train_instance_name = conf['name']+'_'+t+out
                if not conf['separate_cat_ori'] and out != '':
                    continue
                if conf['separate_cat_ori']:
                    if out == '':
                        continue
                    elif out == '_cat':
                        conf['reg_weight'] = 0.0
                        conf['bin_weight'] = 0.0
                        conf['epochs'] = conf['epochs_cat']
                    elif out == '_reg' and conf['enable_reg']:
                        conf['cat_weight'] = 0.0
                        conf['bin_weight'] = 0.0
                        conf['epochs'] = conf['epochs_reg']
                    elif out == '_bin' and conf['enable_bin']:
                        conf['cat_weight'] = 0.0
                        conf['reg_weight'] = 0.0
                        conf['epochs'] = conf['epochs_bin']
                    else:
                        print('UNKNOWN OUTPUT CONFIG {}'.format(out))
                        continue
                if not re.match('sstage_default_sphero_cat', train_instance_name): 
                    print('Skip '+train_instance_name)
                    continue            
                #record_for_type = [e for e in train_records if t in e]
                #print("record_for_type: ", len(record_for_type))
                #assert len(record_for_type) == 1, \
                #       "{} for {} has not one item".format(record_for_type, train_instance_name)
                print("Start training")
                
                if not GPU:
                    p = Process(
                      target=train,
                      args=(train_records[0], conf, ['sphero'], out, r)
                    )
                    p.start() 
                else:
                 
                  train(train_records[0], conf, ['sphero'], out)


  0%|          | 0/2 [00:00<?, ?it/s]

7425
or_conf {'dataset': 'default', 'types': ['copter', 'sphero', 'youbot'], 'epochs_cat': 60, 'epochs_reg': 60, 'epochs_bin': 60, 'optimizer': 'adam', 'learning_rate': 0.0003, 'dropout': 0, 'alpha': 0.5, 'img_size': 35, 'separate_cat_ori': True, 'cat_weight': 1.0, 'reg_weight': 1.0, 'bin_weight': 1.0, 'enable_reg': True, 'enable_bin': True, 'repetions': 4, 'name': 'sstage_default'}
Skip sstage_default_copter_cat
Skip sstage_default_copter_reg
Skip sstage_default_copter_bin
Start training
x
Loaded 59400 crops from ['/content/gdrive/My Drive/data/generated/SecondStage_X/training_rot3.record']
´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´
(59400, 12)
12
(59400, 360)
360
´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´








/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '







5578752/5577668 [==============================] - 1s 0us/step



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






Epoch 00020: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_cat/2020-01-19-19-10-r1/model-20.h5

Epoch 00040: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_cat/2020-01-19-19-10-r1/model-40.h5

Epoch 00060: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_cat/2020-01-19-19-10-r1/model-60.h5
Finished training for sstage_default_sphero
Skip sstage_default_sphero_reg
Skip sstage_default_sphero_bin
Skip sstage_default_youbot_cat
Skip sstage_default_youbot_reg
Skip sstage_default_youbot_bin
Skip sstage_default_copter_cat
Skip sstage_default_copter_reg
Skip sstage_default_copter_bin
Start training
x
Loaded 59400 crops from ['/content/gdrive/My Drive/data/generated/SecondStage_X/training_rot3.record']
´´´´

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '



Epoch 00020: saving model to /content/gdrive/My Drive/SecondStage_colab/output/sstage_default_sphero_cat/2020-01-19-19-14-r1/model-20.h5
